In [1]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

# 1. Extracción de datos

En esta evaluación nos enfrentamos a un desafío emocionante de trabajar en un proyecto real para una empresa que realiza un estudio de universidades por el mundo. El proyecto tiene como objetivo identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.

Para llevar a cabo esta tarea, utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo. Con la ayuda de esta API, podemos acceder a una gran cantidad de datos relevantes, incluyendo el nombre de la universidad, la ciudad donde esta ubicada, el nombre de la institución y otra información importante que nos permitirá llevar a cabo un análisis detallado.

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es: ""http://universities.hipolabs.com/search?country=NOMBREPAIS".

In [2]:
def llamar_api(países):

    """Esta función recibe un único parámetro, que será un string o una lista de strings con el nombre del país o países (en inglés) de los que se quiere
    obtener información. Se realizará la extracción de los datos de los países indicados, y devolverá un DataFrame
    con la información obtenida. En caso de que hubiera algún error, devolverá el número del error y el motivo."""

    if isinstance(países, str): # Comprueba si el parámetro introducido es un string. 

        url = f"http://universities.hipolabs.com/search?country={países}" 

        response = requests.get(url)

        code = response.status_code # Devuelve el código de la extracción de datos

        reason = response.reason # Devuelve el motivo.

        if code == 200: # Comprueba que el código de la extracción sea 200, es decir, que haya ocurrido correctamente.

            print(f"Información de {países} obtenida correctamente, se va a convertir en DataFrame.")

            df =  pd.json_normalize(response.json())

            return df # Devuelve el DataFrame con la información del país indicado. 
        else: 
            return f"Error: {code}, {reason}." # Si no, devuelve el código de error y el motivo.
    
    elif isinstance(países, list):  # Comprueba si el parámetro introducido es una lista.

        df_universidades = pd.DataFrame() #Se crea un DataFrame vacío para ir uniendo los resultados por cada país de la lista introducida.

        for país in países: 

            url = f"http://universities.hipolabs.com/search?country={país}"

            response = requests.get(url) 

            code = response.status_code # Devuelve el código de la extracción de datos

            reason = response.reason # Devuelve el motivo.

            if code == 200: # Comprueba que el código de la extracción sea 200, es decir, que haya ocurrido correctamente.

                print(f"Información de {país} obtenida correctamente, se va a convertir en DataFrame.")

                df =  pd.json_normalize(response.json())

                df_universidades = pd.concat([df_universidades, df], axis = 0) # Se unen los resultados en el DataFrame creado

            else: 
                return f"Error: {code}, {reason}."
            
        
        return df_universidades # Devuelve el DataFrame con la información de los paises indicados si se ha podido crear. Fuera del bucle para
                                # que se pueda completar el bucle. 
    
    else:
        print("Por favor, introduzca un único país o una lista de paises (en inglés).")

In [3]:
lista_paises = ["United States", "Canada", "Argentina"]


df_universidades = llamar_api(lista_paises)


Información de United States obtenida correctamente, se va a convertir en DataFrame.
Información de Canada obtenida correctamente, se va a convertir en DataFrame.
Información de Argentina obtenida correctamente, se va a convertir en DataFrame.


In [4]:
df_universidades.head()

,web_pages,alpha_two_code,state-province,name,domains,country
0,[http://www.marywood.edu],US,None,Marywood University,[marywood.edu],United States
1,[http://www.lindenwood.edu/],US,None,Lindenwood University,[lindenwood.edu],United States
2,[https://sullivan.edu/],US,None,Sullivan University,[sullivan.edu],United States
3,[https://www.fscj.edu/],US,None,Florida State College at Jacksonville,[fscj.edu],United States
4,[https://www.xavier.edu/],US,None,Xavier University,[xavier.edu],United States


In [5]:
df_universidades["country"].unique() #Comprobamos que se haya hecho bien.

array(['United States', 'Canada', 'Argentina'], dtype=object)

# 2. Limpieza

2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

    2.1. Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.

    2.2. La columna de `domains` nos da una información similar a la de `web_pages`. Eliminad la columna domains.

In [6]:
def limpieza_columnas(dataframe):

    """Esta función recibe como parámetro el nombre del dataframe a limpiar.
        Devuelve el dataframe con el nombre de las columnas cambiadas ("-" ahora es "_"), y elimina una columna redundante."""


    nuevas_columnas = {col : col.replace("-", "_") for col in dataframe.columns} # Se crea un diccionario con los nombres antiguos como key, y 
                                                                                #los nuevos nombres como value.

    dataframe.rename(columns = nuevas_columnas, inplace = True) # Se realiza el cambio del nombre de las columnas.
    
    dataframe.drop("domains", axis = 1, inplace = True) # Se elimina la columna de domains.

    return dataframe # Se devuelve el dataframe con los cambios aplicados

In [7]:
limpieza_columnas(df_universidades)

,web_pages,alpha_two_code,state_province,name,country
0,[http://www.marywood.edu],US,None,Marywood University,United States
1,[http://www.lindenwood.edu/],US,None,Lindenwood University,United States
2,[https://sullivan.edu/],US,None,Sullivan University,United States
3,[https://www.fscj.edu/],US,None,Florida State College at Jacksonville,United States
4,[https://www.xavier.edu/],US,None,Xavier University,United States
...,...,...,...,...,...
167,[http://www.untref.edu.ar/],AR,Buenos Aires,Universidad Nacional de Tres de Febrero,Argentina
168,[http://www.utdt.edu/],AR,Ciudad Autónoma de Buenos Aires,Universidad Torcuato di Tella,Argentina
169,[http://www.utn.edu.ar/],AR,Ciudad Autónoma de Buenos Aires,Universidad Tecnológica Nacional,Argentina
170,[http://www.vaneduc.edu.ar/uai/],AR,Ciudad Autónoma de Buenos Aires,Universidad Abierta Interamericana,Argentina


3. Si exploramos la columna de `web_pages`, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de `web_pages` tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejercicio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. [Aquí](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) tenéis la documentación de este método.

In [8]:
def limpieza_webs(dataframe, columna):

    """Esta función recibe dos parámetros, uno es el nombre del dataframe y el otro es el nombre de la columna de la que se desea hacer el explode.
    Devuelve el dataframe con estos cambios realizados. """

    return dataframe.explode(columna)

In [9]:
df_universidades.shape #Comprobamos el número de filas, para poder ver luego si se ha ejecutado bien el cambio. 

(5008, 5)

In [10]:
df_universidades = limpieza_webs(df_universidades, "web_pages") # Se aplica la función al dataframe. 

In [11]:
df_universidades.shape # Comprobamos que se ha ejecutado el cambio y que haya más dilas que antes. 

(5034, 5)

4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [12]:
def eliminar_duplicados(dataframe, columna):

    """Esta función recibe dos parámetros, uno es el nombre del dataframe y el otro es el nombre de la columna de la que se desea eliminar
        los duplicados.
        Comprueba si hay dos duplicados y, si los hay, los elimina, devolviendo el dataframe sin ellos. Si, por el contrario, no hay
        duplicados, devuelve un mensaje donde se avisa al usuario de que no hay duplicados en esa columna. """
        

    if dataframe.duplicated(subset = columna).sum() > 0: #Se comprueba que haya duplicados.

        print(f"Hay {dataframe.duplicated(subset = columna).sum()} duplicados en {columna}. Vamos a eliminarlos.") 

        dataframe.drop_duplicates(subset = columna, inplace = True) # Se eliminan los duplicados de la columna especificada. 

        print(f"¡Listo! Ahora hay {dataframe.duplicated(subset = columna).sum()} duplicados en la columna {columna}.") # Se comprueba que el proceso se haya hecho bien.
    
        return dataframe #Se devuelve el dataframe sin duplicados.
    else:
        return f"No hay duplicados en la columna {columna}." #Si no hay duplicados, se avisa al usuario.


In [13]:
eliminar_duplicados(df_universidades, "name")

Hay 2543 duplicados en name. Vamos a eliminarlos.
¡Listo! Ahora hay 0 duplicados en la columna name.


,web_pages,alpha_two_code,state_province,name,country
0,http://www.marywood.edu,US,None,Marywood University,United States
1,http://www.lindenwood.edu/,US,None,Lindenwood University,United States
2,https://sullivan.edu/,US,None,Sullivan University,United States
3,https://www.fscj.edu/,US,None,Florida State College at Jacksonville,United States
4,https://www.xavier.edu/,US,None,Xavier University,United States
...,...,...,...,...,...
81,http://www.untref.edu.ar/,AR,Buenos Aires,Universidad Nacional de Tres de Febrero,Argentina
82,http://www.utdt.edu/,AR,Ciudad Autónoma de Buenos Aires,Universidad Torcuato di Tella,Argentina
83,http://www.utn.edu.ar/,AR,Ciudad Autónoma de Buenos Aires,Universidad Tecnológica Nacional,Argentina
84,http://www.vaneduc.edu.ar/uai/,AR,Ciudad Autónoma de Buenos Aires,Universidad Abierta Interamericana,Argentina


5. Si exploramos la columna de `state_province` veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [14]:
df_universidades["state_province"].isnull().sum() # Primero, comprobamos el número de nulos.

2204

In [15]:
def reemplazar_nulos(columna):
    """Esta función recibe un parámetro, que es la columna de la que se quieren cambiar `None` a `np.nan`. 
    Devuelve los nulos cambiados en dicha columna."""

    if columna == None:
        return np.nan
    else: 
        return columna

In [16]:
def aplicar_nulos(dataframe, columna):

    """Esta función recibe dos parámetros, uno es el dataframe y otro la columna sobre la que se quiere aplicar la función `reemplazar_nulos`,
    que reemplaza los nulos tipo `None` por `np.nan`.
    Devuelve el dataframe con estos cambios aplicados."""

    dataframe[columna] = dataframe[columna].apply(reemplazar_nulos)
    return dataframe

In [17]:
aplicar_nulos(df_universidades, "state_province")

,web_pages,alpha_two_code,state_province,name,country
0,http://www.marywood.edu,US,NaN,Marywood University,United States
1,http://www.lindenwood.edu/,US,NaN,Lindenwood University,United States
2,https://sullivan.edu/,US,NaN,Sullivan University,United States
3,https://www.fscj.edu/,US,NaN,Florida State College at Jacksonville,United States
4,https://www.xavier.edu/,US,NaN,Xavier University,United States
...,...,...,...,...,...
81,http://www.untref.edu.ar/,AR,Buenos Aires,Universidad Nacional de Tres de Febrero,Argentina
82,http://www.utdt.edu/,AR,Ciudad Autónoma de Buenos Aires,Universidad Torcuato di Tella,Argentina
83,http://www.utn.edu.ar/,AR,Ciudad Autónoma de Buenos Aires,Universidad Tecnológica Nacional,Argentina
84,http://www.vaneduc.edu.ar/uai/,AR,Ciudad Autónoma de Buenos Aires,Universidad Abierta Interamericana,Argentina


In [18]:
df_universidades["state_province"].isnull().sum() # Comprobamos que el número de nulos sea el mismo y ver que se hayan aplicado bien los cambios. 

2204

6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [19]:
def imputar_nulos(dataframe, columna):

    """Esta función recibe dos parámetros, uno es el dataframe y otro la columna sobre la que se quiere aplicar la imputación de nulos.
    Devuelve el dataframe con estos cambios realizados. """

    return dataframe[columna].fillna(value = "Unknown", inplace = True)

In [20]:
imputar_nulos(df_universidades, "state_province")

In [21]:
df_universidades["state_province"].isnull().sum() #Se comprueba que se haya hecho bien.

0

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso, [aquí](https://pypi.org/project/geopy/) la documentación. Para desarrollar este ejercicio deberéis:
- Sacar los valores únicos de la columna state_province.

- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:
    - NV: reemplazalo por Nevada
    - TX: reemplazalo por Texas
    - IN: reemplazalo por Indianapolis
    - CA: reemplazalo por California
    - VA: reemplazalo por Virginia
    - NY: reemplazalo por New York
    - MI: reemplazalo por Michigan
    - GA: reemplazalo por Georgia
    - ND: reemplazalo por North Dakota

- Otros valores que tenemos más formateados son y que deberemos reemplazar:

    - New York, NY. Deberéis reemplazarlo por "New York".
    - 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires".


- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

- Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [22]:
df_universidades["state_province"].unique() #Primero, sacamos los valores unicos de la columna.

array(['Unknown', 'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA',
       'TX', 'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA',
       'Illinois', 'New Hampshire', 'North Carolina', 'South Carolina',
       'Virginia', 'Washington', 'Missouri', 'California', 'NY', 'ND',
       'MI', 'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

In [23]:
def reemplazar_estados(elemento): 

    """Esta función recibe un único parámetro, que es cada uno de los elementos de una columna, y se realizan los cambios indicados.
    Devuelve el elemento con los cambios aplicados si cumple las condiciones, o el elemento sin cambios."""

    if elemento == "NV":
        return "Nevada"
    elif elemento == "TX":
        return "Texas"
    elif elemento == "IN":
        return "Indianapolis"
    elif elemento == "CA":
        return "California"
    elif elemento == "VA":
        return "Virginia"
    elif elemento == "NY" or elemento == "New York, NY":
        return "New York"
    elif elemento == "MI":
        return "Michigan"
    elif elemento == "GA":
        return "Georgia"
    elif elemento == "ND":
        return "North Dakota"
    elif elemento == "Ciudad Autónoma de Buenos Aires":
        return "Buenos Aires"
    else:
        return elemento

In [24]:
def aplicar_reemplazo(dataframe, columna):
    """Esta función recibe dos parámetros, que es el nombre del dataframe y el nombre de la columna sobre la que se quieren aplicar los cambios de la 
    función `reemplazar_estados`. Devuelve el dataframe con los estados cambiados, en caso de que apliquen las condiciones."""

    dataframe[columna] = dataframe[columna].apply(reemplazar_estados)

    return dataframe

In [25]:
aplicar_reemplazo(df_universidades, "state_province")

,web_pages,alpha_two_code,state_province,name,country
0,http://www.marywood.edu,US,Unknown,Marywood University,United States
1,http://www.lindenwood.edu/,US,Unknown,Lindenwood University,United States
2,https://sullivan.edu/,US,Unknown,Sullivan University,United States
3,https://www.fscj.edu/,US,Unknown,Florida State College at Jacksonville,United States
4,https://www.xavier.edu/,US,Unknown,Xavier University,United States
...,...,...,...,...,...
81,http://www.untref.edu.ar/,AR,Buenos Aires,Universidad Nacional de Tres de Febrero,Argentina
82,http://www.utdt.edu/,AR,Buenos Aires,Universidad Torcuato di Tella,Argentina
83,http://www.utn.edu.ar/,AR,Buenos Aires,Universidad Tecnológica Nacional,Argentina
84,http://www.vaneduc.edu.ar/uai/,AR,Buenos Aires,Universidad Abierta Interamericana,Argentina


In [26]:
df_universidades["state_province"].unique() # Comprobamos que se haya realizado correctamente. 

array(['Unknown', 'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa',
       'Virginia', 'Indiana', 'Colorado', 'Ohio', 'Indianapolis',
       'New York', 'California', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Washington', 'Missouri',
       'North Dakota', 'Michigan', 'Florida', 'Georgia', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Santiago Del Estero', 'Misiones', 'Catamarca',
       'Formosa', 'Jujuy', 'La Rioja', 'La Pampa', 'San Juan', 'San Luis',
       'Tucumán'], dtype=object)

> Opté por realizar la parte de limpieza en funciones pequeñas para poder seguir los ejercicios paso a paso y de una manera algo más "universal". No obstante, a la hora de crear la clase, solamente hay un único método de limpieza que agrupa todos los pasos en una sola función, ya que se entiende que cada vez que se llame a esta API se realizarán siempre los mismos pasos de limpieza. 

>La función quedaría así definida:


In [27]:
def limpieza(self, dataframe):

    """Esta función recibe como parámetro el nombre del dataframe a limpiar.
        Devuelve el dataframe con el nombre de las columnas cambiadas ("-" ahora es "_"), y elimina una columna redundante. Además,
        realiza el explode a la columna "web_pages, elimina los duplicados de la columna  "name", imputa los nulos de
        la columna "state_province" por la categoría "Unknown". Asimismo, cambia los estados por el nombre completo. 
        Devuelve el dataframe con los datos aplicados. """

    nuevas_columnas = {col : col.replace("-", "_") for col in dataframe.columns} # Se crea un diccionario con los nombres antiguos como key, y 
                                                                                    #los nuevos nombres como value.

    dataframe.rename(columns = nuevas_columnas, inplace = True) # Se realiza el cambio del nombre de las columnas.
            
    dataframe.drop("domains", axis = 1, inplace = True) # Se elimina la columna de domains.

    dataframe = dataframe.explode("web_pages") # Se realiza el explode para la columna de "web_pages"

    dataframe.drop_duplicates(subset = "name", inplace = True) # Se eliminan los duplicados en la columna "name"

    dataframe["state_province"].fillna("Unknown", inplace = True)

    dataframe["state_province"] = dataframe["state_province"].replace("NV", "Nevada").replace("TX", "Texas").replace("IN", "Indianapolis").replace("CA", "California").replace("VA", "Virginia").replace("NY", "New York")
            
    dataframe["state_province"] = dataframe["state_province"].replace("New York, NY", "New York").replace("MI", "Michigan").replace("GA", "Georgia").replace("ND", "North Dakota").replace("Ciudad Autónoma de Buenos Aires", "Buenos Aires")

        # Se realizan varias modificaciones a la columna "state_province", imputando los nulos por una nueva categoría y reemplazando los nombres de estados
        # por otros nombres más claros. El replace se podría haber hecho una única línea de código, pero quedaba muy larga, así que he 
        # optado por partirla en dos. 

    return dataframe # Se devuelve el dataframe con los cambios

In [28]:
def sacar_coordenadas(dataframe, columna):
    """Esta función recibe dos parámetros, que es el nombre del dataframe y el nombre de la columna sobre la que se quieren sacar las coordenadas.
    Se utiliza la librería de Geopy para conseguir la latitud y la longitud de los lugares indicados. 
    Devuelve el DataFrame con dos columnas nuevas, "lat" y "long", que contienen las coordenadas en función del lugar de la columna indicada."""

    lista_estados = list(dataframe[columna].unique()) # Primero, se crea una lista con los valores únicos de esa columna.

    df_localizacion = pd.DataFrame(columns = ["state_province", "lat", "long"]) # Se crea un datagrame vacío para poder introducir los datos. 

    for estado in lista_estados: 

        if estado != "Unknown":  
            geo = Nominatim(user_agent = "nombre")
            localizacion = geo.geocode(f"{estado}")
            state = f"{estado}"
            latitud = localizacion[1][0]
            longitud = localizacion[1][1]

            df_localizacion.loc[len(df_localizacion.index)] = [state, latitud, longitud] #Se añaden los datos sacados de Geopy en la última fila del DataFrame creado.
        
        else: # Si el estado/lugar entra en la categoría creada previamente de "Unknown", entonces las coordenadas se convertirán a un nulo de NumPY.
            
            state = f"{estado}"
            latitud = np.nan
            longitud = np.nan
            
            df_localizacion.loc[len(df_localizacion.index)] = [state, latitud, longitud]
        
        
    dataframe = pd.merge(dataframe, df_localizacion, on = columna, how = "left")

    return dataframe


In [29]:
sacar_coordenadas(df_universidades, "state_province")

,web_pages,alpha_two_code,state_province,name,country,lat,long
0,http://www.marywood.edu,US,Unknown,Marywood University,United States,NaN,NaN
1,http://www.lindenwood.edu/,US,Unknown,Lindenwood University,United States,NaN,NaN
2,https://sullivan.edu/,US,Unknown,Sullivan University,United States,NaN,NaN
3,https://www.fscj.edu/,US,Unknown,Florida State College at Jacksonville,United States,NaN,NaN
4,https://www.xavier.edu/,US,Unknown,Xavier University,United States,NaN,NaN
...,...,...,...,...,...,...,...
2486,http://www.untref.edu.ar/,AR,Buenos Aires,Universidad Nacional de Tres de Febrero,Argentina,-34.607568,-58.437089
2487,http://www.utdt.edu/,AR,Buenos Aires,Universidad Torcuato di Tella,Argentina,-34.607568,-58.437089
2488,http://www.utn.edu.ar/,AR,Buenos Aires,Universidad Tecnológica Nacional,Argentina,-34.607568,-58.437089
2489,http://www.vaneduc.edu.ar/uai/,AR,Buenos Aires,Universidad Abierta Interamericana,Argentina,-34.607568,-58.437089


8. Crea una BBDD en mysql:


El script de la creación de la base de datos se encuentra en `../sql/script_bbdd_universidades.sql`. El esquema resultante es el siguiente:

![Esquema de la BBDD](../sql/imagen_esquema_bbdd.png)

9. Introduce todo el código que habéis ido creando en funciones.

Ya está el código realizado en funciones. 